In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# Hyper-parameter
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='./data', 
                                           train=True, 
                                           transform=transforms.ToTensor(), 
                                           download=True)
test_dataset = torchvision.datasets.MNIST(root='./data',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [3]:
# Fully connected neural netword with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        # call __init__ method of parent class to initialize the class
        super(NeuralNet, self).__init__()  # super().__init__() in python 3
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out
    
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [4]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                 .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.5346
Epoch [1/5], Step [200/600], Loss: 0.2261
Epoch [1/5], Step [300/600], Loss: 0.2837
Epoch [1/5], Step [400/600], Loss: 0.2072
Epoch [1/5], Step [500/600], Loss: 0.1901
Epoch [1/5], Step [600/600], Loss: 0.1083
Epoch [2/5], Step [100/600], Loss: 0.2890
Epoch [2/5], Step [200/600], Loss: 0.0717
Epoch [2/5], Step [300/600], Loss: 0.0388
Epoch [2/5], Step [400/600], Loss: 0.0782
Epoch [2/5], Step [500/600], Loss: 0.0873
Epoch [2/5], Step [600/600], Loss: 0.0764
Epoch [3/5], Step [100/600], Loss: 0.0827
Epoch [3/5], Step [200/600], Loss: 0.0601
Epoch [3/5], Step [300/600], Loss: 0.0580
Epoch [3/5], Step [400/600], Loss: 0.0704
Epoch [3/5], Step [500/600], Loss: 0.0981
Epoch [3/5], Step [600/600], Loss: 0.0772
Epoch [4/5], Step [100/600], Loss: 0.1514
Epoch [4/5], Step [200/600], Loss: 0.0252
Epoch [4/5], Step [300/600], Loss: 0.0740
Epoch [4/5], Step [400/600], Loss: 0.0383
Epoch [4/5], Step [500/600], Loss: 0.0138
Epoch [4/5], Step [600/600], Loss:

In [5]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)  # At axis=1. predicted = torch.max()[1], the index of max value
        total = labels.size(0)
        correct = (predicted == labels).sum().item()
        
    print('Accuracy of the network on the 10000 test images: {}%'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Accuracy of the network on the 10000 test images: 99.0%
